In [1]:
import nbimporter
from template_manager import template_selector
from patient_data_extraction import extract_patient_info,extract_patient_measurements,image_extractor,mot_extractor
from file_conversion import cargar_path
from aux_calculations import conv_vel_grad,calc_e_e,list_and_len_checker,convert_to_float_and_round
from docx import Document
from docxtpl import DocxTemplate

def render_template(path)-> None:
    '''
    acepta un path y determina si es cardiaco o stress, si son estos extrae medidas, tratornos de motilidad y mapa polar
    Si no lo es extrae solamente datos del paciente y las imagenes del archivo docx producido por el equipo vinno
    guarda la renderizaci'on en la misma carpeta donde estaba el docx para poder realizar una comparacion
    
    '''
    doc=Document(path)
    template,tipo=template_selector(path)
    info_pac= extract_patient_info(doc)
    image=image_extractor(doc,template,tipo=tipo)
    #guardo el save path con nombre tipo de estudio y fecha en la misma carpeta.
    save_path=str(path).rsplit('\\',1)[0]+f'\\{info_pac["Name"]} {tipo} {info_pac["Exam Date"]}.docx'
    print(save_path)
    print(tipo)
    #Si es carotida solo van los datos del paciente
    if tipo in ['card','stress']: 
        medidas= extract_patient_measurements(path)
        #calculo gradiente VDAD
        medidas['grad_vd_ad']= max([conv_vel_grad(i,'cm') for i in medidas['TR Vmax']])
    
        if list_and_len_checker('TR Vmax',medidas):
            medidas['TR Vmax1']= convert_to_float_and_round(medidas['TR Vmax'][0])
            medidas['TR Vmax2']=convert_to_float_and_round(medidas['TR Vmax'][1])
        if tipo =='stress':
            mot=mot_extractor(doc)
            print(medidas['Med Vel E'])
            calc_e_e(medidas)
            context={**info_pac,**medidas,'image': image['image'], 'mot': mot['mot']}
        #genero context
        
        else:
            context={**info_pac,**medidas,'image': image['image']}
    else: 
        context={**info_pac,'image': image['image']}
        
    #quito los espacios en los context para que no haya conflicto
    context=(
        {key.replace(' ', '_').replace('/','_').replace('(','_').replace(')','_').
            replace('-','_'): value 
            for key, value in context.items()}
            )

    template.render(context)
    #template.save(save_path)
    print(f'informe {info_pac["Name"]} creado con exito')
        

In [2]:
p=r"C:\Users\Luciano\Documents\temp\[albertodec rolando]-[]-[8c7e13cd-902a-46f2-8beb-df60ff1f8ab6]"
p=cargar_path(p)
p[0]

WindowsPath('C:/Users/Luciano/Documents/temp/[albertodec rolando]-[]-[8c7e13cd-902a-46f2-8beb-df60ff1f8ab6]/7_S1-6P_Lucho Card.Report.V3.docx')